Activate new_scanpy with the following before opening this notebook:
conda activate new_scanpy

Then, run: 
jupyter lab

We will use the Kang dataset, which is a 10x droplet-based scRNA-seq peripheral blood mononuclear cell (PBMC) data from 8 Lupus patients before and after 6h-treatment with INF-β (16 samples in total)[Kang et al., 2018].

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random

import scanpy as sc

In [ ]:
import os
import pickle as pkl

from pydeseq2.dds import DeseqDataSet
from pydeseq2.default_inference import DefaultInference
from pydeseq2.ds import DeseqStats
import gseapy

In [ ]:
#feel free to change these settings according to your needs
sc.set_figure_params(dpi=80, figsize=(4,4))
sc.settings.verbosity=0
plt.rcParams['figure.dpi'] = 80
plt.rcParams['figure.figsize'] = (4, 4)

In [ ]:
#we are just going to use the Kang dataset from pertpy package 
import pertpy

In [ ]:
adata = pertpy.data.kang_2018()
adata

In [ ]:
sc.pl.umap(adata, color=["label", "cell_type"])

In [ ]:
#filtering out cells and genes
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)
adata

In [ ]:
#creating a patient-condition column
adata.obs["sample"] = [
    f"{rep}_{l}" for rep, l in zip(adata.obs["replicate"], adata.obs["label"])
]

In [ ]:
adata.obs

We run the pipeline on CD14+ Monocytes subset of the data, as it was shown in the paper that the highest number of DE genes was identified in this subpopulation.

In [ ]:
#subset CD14+ Monocytes
mono=adata[adata.obs["cell_type"].isin(["CD14+ Monocytes"])]
mono

In [ ]:
#aggregate data matrix based on sample to create pseudobalks
mono_agg=sc.get.aggregate(mono, by="sample", func=['sum'])

In [ ]:
mono_agg

In [ ]:
mono_agg.X=mono_agg.layers["sum"]

In [ ]:
mono_agg.obs

In [ ]:
mono_agg.X

In [ ]:
#PyDESeq2 needs a Pandas DataFrame of count matrix with obs_names as row names and var_names as column names
counts_df = pd.DataFrame(mono_agg.X, index=mono_agg.obs_names, columns=mono.var_names)
counts_df

In [ ]:
additional_metadata = {
    'patient_1015_ctrl': 'ctrl',
    'patient_1015_stim': 'stim',
    'patient_1016_ctrl':'ctrl',
    'patient_1016_stim':'stim',
    'patient_101_ctrl':'ctrl',
    'patient_101_stim':'stim',
    'patient_1039_ctrl':'ctrl',
    'patient_1039_stim':'stim',
    'patient_107_ctrl':'ctrl',
    'patient_107_stim':'stim',
    'patient_1244_ctrl':'ctrl',
    'patient_1244_stim':'stim',
    'patient_1256_ctrl':'ctrl',
    'patient_1256_stim':'stim',
    'patient_1488_ctrl':'ctrl',
    'patient_1488_stim':'stim'
    # Add more samples as needed
}

# Add this metadata as a new column in the .obs DataFrame
mono_agg.obs['label'] = mono_agg.obs.index.map(additional_metadata)

#you also use loop:
#patient_ids = [1015, 1016, 101, 1039, 107, 1244, 1256, 1488]
#labels = ['ctrl', 'stim'] 
#additional_metadata = {f'patient_{id}_{label}': label for id in patient_ids for label in labels}
#mono_agg.obs['label'] = mono_agg.obs.index.map(additional_metadata)


In [ ]:
mono_agg

In [ ]:
additional_metadata02 = {
    'patient_1015_ctrl': 'patient_1015',
    'patient_1015_stim': 'patient_1015',
    'patient_1016_ctrl':'patient_1016',
    'patient_1016_stim':'patient_1016',
    'patient_101_ctrl':'patient_101',
    'patient_101_stim':'patient_101',
    'patient_1039_ctrl':'patient_1039',
    'patient_1039_stim':'patient_1039',
    'patient_107_ctrl':'patient_107',
    'patient_107_stim':'patient_107',
    'patient_1244_ctrl':'patient_1244',
    'patient_1244_stim':'patient_1244',
    'patient_1256_ctrl':'patient_1256',
    'patient_1256_stim':'patient_1256',
    'patient_1488_ctrl':'patient_1488',
    'patient_1488_stim':'patient_1488'
    # Add more samples as needed
}

# Add this metadata as a new column in the .obs DataFrame
mono_agg.obs['replicate'] = mono_agg.obs.index.map(additional_metadata02)


In [ ]:
mono_agg.obs

In [ ]:
metadata = sc.get.obs_df(mono_agg, keys = ["label", "replicate"])

In [ ]:
samples_to_keep = ~metadata.label.isna()
counts_df = counts_df.loc[samples_to_keep]
metadata = metadata.loc[samples_to_keep]

In [ ]:
genes_to_keep = counts_df.columns[counts_df.sum(axis=0) >= 10]
counts_df = counts_df[genes_to_keep]

In [ ]:
#creating a DeseqDataSet object
inference = DefaultInference(n_cpus=8)
dds = DeseqDataSet(
    counts=counts_df,
    metadata=metadata,
    design_factors="label",
    refit_cooks=True,
    inference=inference,
)

In [ ]:
#running deseq2() method to fit dispersions and LFCs
dds.deseq2()

In [ ]:
print(dds)

In [ ]:
print(dds.varm["dispersions"])

In [ ]:
print(dds.varm["LFC"])

In [ ]:
#statistical analysis
stat_res = DeseqStats(dds, inference=inference)

In [ ]:
stat_res.summary()

In [ ]:
results=stat_res.results_df
results

performing over-representation analysis (Enrichr API). 
This requires subsetting up or down regulated genes resulted from DE gene analysis

In [ ]:
#significant differentially expressed genes
sig_results=results[results["padj"]< 0.01]
sig_results

subset genes that are more expressed (positive log2 fold change) in stim vs. ctrl

In [ ]:
#upregulated genes in stim condition
pos_sig_results=sig_results[sig_results["log2FoldChange"]>0]
pos_sig_results

In [ ]:
gseapy.get_library_name(organism="Human")

In [ ]:
enrichr_result=gseapy.enrichr(gene_list=pos_sig_results.index.tolist(), gene_sets=["GO_Biological_Process_2023"], 
                          organism="Human",cutoff=0.05)
enrichr_result.results.head()

In [ ]:
top_results_enrichr=enrichr_result.results.sort_values(by="Adjusted P-value")[0:10]
top_results_enrichr

In [ ]:
ax=gseapy.barplot(enrichr_result.results, column="Adjusted P-value", group="Gene set", size=10,
                  top_term=10)

plt.show()

performing gene set enrichment analysis 

In [ ]:
mono.obs['stim'] = pd.Categorical(mono.obs['label'], categories=["stim", "ctrl"], ordered=True)

In [ ]:
indices = mono.obs.sort_values(['stim']).index
mono = mono[indices,:]

In [ ]:
mono.obs

In [ ]:
import time
t1 = time.time()
res = gseapy.gsea(data=mono.to_df().T, # row -> genes, column-> samples
        gene_sets="GO_Biological_Process_2023",
        cls=mono.obs.stim,
        permutation_num=100, #increase the number of permutation to e.g. 1000 for your analysis
        permutation_type='phenotype',
        outdir=None,
        method='s2n', # signal_to_noise
        threads= 8)
t2=time.time()
print(t2-t1)

In [ ]:
res.res2d

In [ ]:
top_results_gsea=res.res2d.sort_values(by="NES", ascending=False)[0:10]
top_results_gsea

In [ ]:
plt.figure(figsize=(10, 8))
plt.barh(top_results_gsea["Term"], top_results_gsea['NES'], color='skyblue')
plt.xlabel('Normalized Enrichment Score (NES)')
plt.ylabel('Term')
plt.title('GSEA Normalized Enrichment Scores')
plt.gca().invert_yaxis()  # Highest NES at the top
plt.show()

In [ ]:
## Heatmap of gene expression
i = 0
genes = res.res2d.Lead_genes.iloc[i].split(";")
ax = gseapy.heatmap(df = res.heatmat.loc[genes],
           z_score=None,
           title=res.res2d.Term.iloc[i],
           figsize=(6,5),
           cmap=plt.cm.viridis,
           xticklabels=False)

In [ ]:
term = res.res2d.Term
axs = res.plot(terms=term[:5])